In [3]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random

import json
with open('good_intent.json') as file:
    good_data = json.load(file)

with open('bad_intent.json') as file:
    bad_data = json.load(file)

    
good_words = []
good_labels = []
bad_words = []
bad_labels = []
good_docs_x = []
good_docs_y = []
bad_docs_x = []
bad_docs_y = []

for intent in good_data['intents']:
    for pattern in intent['patterns']:
        good_wrds = nltk.word_tokenize(pattern)
        good_words.extend(good_wrds)
        good_docs_x.append(good_wrds)
        good_docs_y.append(intent["tag"])
        
    if intent['tag'] not in good_labels:
        good_labels.append(intent['tag'])
        
for intent in bad_data['intents']:
    for pattern in intent['patterns']:
        bad_wrds = nltk.word_tokenize(pattern)
        bad_words.extend(bad_wrds)
        bad_docs_x.append(bad_wrds)
        bad_docs_y.append(intent["tag"])
        
    if intent['tag'] not in bad_labels:
        bad_labels.append(intent['tag'])

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)

NameError: name 'words' is not defined

In [17]:
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 2999  | total loss: 0.33710 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.33710 - acc: 0.9660 -- iter: 16/21
Training Step: 3000  | total loss: 0.30613 | time: 0.011s
| Adam | epoch: 1000 | loss: 0.30613 - acc: 0.9694 -- iter: 21/21
--
INFO:tensorflow:C:\Users\Elsa-\Downloads\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [20]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

intent_num = 1;

def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        if(intent_num == 1):
            for tg in good_data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print(random.choice(responses))
        else:
            for tg in bad_data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: picture
Yes, the picture was nice!
You: pic
I learned alot of new information aswell!
You: pic
Glad you found the content good!


KeyboardInterrupt: Interrupted by user